In [8]:
# Parameters
name = "Gülnarə"
surname = "Əzizova"
sector = "S\u0259hhiyy\u0259"
field = "V\u0259r\u0259m X\u0259st\u0259liyi"
start_year = 2024
end_year = 2026

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import matplotlib
matplotlib.use('Agg')
warnings.filterwarnings('ignore')

In [10]:
# Prophet model (alternative implementation)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    PROPHET_AVAILABLE = False
    print("Prophet not available, using alternative forecasting methods")

In [11]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as patches
from datetime import datetime, timedelta

In [12]:
# Set matplotlib to use available fonts
plt.rcParams['font.family'] = ['DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
# Enable Unicode support
plt.rcParams['axes.unicode_minus'] = False

# Vərəm Xəstəliyi üzrə Statistik Göstəricilər

In [13]:
# Read the CSV data
def read_tuberculosis_data():
  df = pd.read_csv('vərəm_xəstəliyi.csv')
  df['Illər'] = pd.to_datetime(df['Illər'], format='%Y')
  df.set_index('Illər', inplace=True)
  return df

read_tuberculosis_data()

,0-13 yaşlı - cəmi,14-17 yaşlı - cəmi,18-29 yaşlı - cəmi,30-44 yaşlı - cəmi,45-64 yaşlı - cəmi,65 və yuxarı yaşda - cəmi,kişilər,qadınlar,"İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər",Əhalinin hər 100 000 nəfərinə -cəmi (müvafiq cins və yaş qruplarına görə)
Illər,,,,,,,,,,
2007-01-01,92.050000,126.600000,776.050000,599.750000,315.850000,60.100000,433.562753,136.171429,3713.0,43.159865
2008-01-01,100.016634,142.289073,867.859433,670.108696,403.743739,72.631222,490.868551,161.488810,4255.0,48.804267
2009-01-01,104.280000,137.069701,946.170256,598.436538,438.801331,110.808404,498.161770,177.003565,4401.0,49.856695
2010-01-01,128.511876,177.864729,1025.183504,592.425987,518.694157,104.553800,541.633383,194.472762,4801.0,53.736723
2011-01-01,115.273185,174.414483,984.476252,592.129676,567.417206,134.336858,519.922345,222.227760,4836.0,53.418166
2012-01-01,94.659109,161.861842,902.590788,583.366245,579.988930,127.604732,505.300173,202.668085,4616.0,50.306240
2013-01-01,94.051696,189.465368,856.030377,554.654704,593.160612,117.716770,501.591213,193.030449,4528.0,48.704931
2014-01-01,77.168031,149.424973,821.245267,537.982693,622.098533,116.609392,484.695214,186.618678,4384.0,46.563499
2015-01-01,75.524441,133.606427,708.768452,516.553939,558.079978,123.559997,423.283958,187.704918,3989.0,41.860367


In [14]:
def linear_trend_forecast_1(series, periods=3):
    X = np.arange(len(series)).reshape(-1, 1)
    y = series.values

    model_linear = LinearRegression()
    model_linear.fit(X, y)

    future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
    forecast_linear = model_linear.predict(future_X)

    # Simple confidence interval estimation
    residuals_linear = y - model_linear.predict(X)
    std_error_linear = np.std(residuals_linear)
    conf_int_linear = np.column_stack([forecast_linear - 1.96*std_error_linear, forecast_linear + 1.96*std_error_linear])

    return forecast_linear, conf_int_linear, model_linear

In [15]:
# Time series forecasting - ARIMA
def arima_forecast_1(series, periods=3, order=(1,1,1)):
    try:
        model_arima = ARIMA(series, order=order)
        fitted_model = model_arima.fit()
        forecast_arima = fitted_model.forecast(steps=periods)
        conf_int_arima = fitted_model.get_forecast(steps=periods).conf_int()
        return forecast_arima.values, conf_int_arima.values, fitted_model
    except:
        try:
            # Fallback to simple ARIMA
            model_arima = ARIMA(series, order=(1,1,0))
            fitted_model = model_arima.fit()
            forecast_arima = fitted_model.forecast(steps=periods)
            conf_int_arima = fitted_model.get_forecast(steps=periods).conf_int()
            return forecast_arima.values, conf_int_arima.values, fitted_model
        except:
            # Final fallback to linear trend
            return linear_trend_forecast_1(series, periods)

In [16]:
# Time series forecasting - Prophet
def prophet_forecast_1(series, periods=3):
    if not PROPHET_AVAILABLE:
        return linear_trend_forecast_1(series, periods)
    try:
        df = pd.DataFrame({
            'ds': series.index,
            'y': series.values
        })

        model_prophet = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        model_prophet.fit(df)

        future = model_prophet.make_future_dataframe(periods=periods, freq='Y')
        forecast_prophet = model_prophet.predict(future)

        forecast_values_prophet = forecast_prophet.tail(periods)['yhat'].values
        conf_int_prophet = forecast_prophet.tail(periods)[['yhat_lower', 'yhat_upper']].values

        return forecast_values_prophet, conf_int_prophet, model_prophet
    except:
        return linear_trend_forecast_1(series, periods)

In [17]:
def random_forest_forecast_1(series, periods=3):
    # Create features (lagged values)
    n_lags = min(5, len(series) // 2)
    X, y = [], []

    for i in range(n_lags, len(series)):
        X.append(series.values[i-n_lags:i])
        y.append(series.values[i])

    X, y = np.array(X), np.array(y)

    model_rfr = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rfr.fit(X, y)

    # Generate forecasts
    forecasts_rfr = []
    last_values_rfr = series.values[-n_lags:]

    for _ in range(periods):
        pred_rfr = model_rfr.predict([last_values_rfr])[0]
        forecasts_rfr.append(pred_rfr)
        last_values_rfr = np.append(last_values_rfr[1:], pred_rfr)

    return np.array(forecasts_rfr), None, model_rfr

In [18]:
# Analysis functions
def perform_tuberculosis_analysis():
  df = read_tuberculosis_data()
  groups = [
      '0-13 yaşlı - cəmi',
      '14-17 yaşlı - cəmi',
      '18-29 yaşlı - cəmi',
      '30-44 yaşlı - cəmi',
      '45-64 yaşlı - cəmi',
      '65 və yuxarı yaşda - cəmi',
      'kişilər',
      'qadınlar',
      'İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər',
      'Əhalinin hər 100 000 nəfərinə -cəmi (müvafiq cins və yaş qruplarına görə)'
      ]

  results = {}
  for group in groups:
    series = df[group]
    # ARIMA forecast
    arima_pred, arima_conf, arima_model = arima_forecast_1(series)

    # Prophet forecast
    prophet_pred, prophet_conf, prophet_model = prophet_forecast_1(series)

    # Random Forest forecast
    rf_pred, rf_conf, rf_model = random_forest_forecast_1(series)

    # Linear trend forecast
    linear_pred, linear_conf, linear_model = linear_trend_forecast_1(series)

    results[group] = {
        'historical': series,
        'arima': {'forecast': arima_pred, 'conf_int': arima_conf, 'model': arima_model},
        'prophet': {'forecast': prophet_pred, 'conf_int': prophet_conf, 'model': prophet_model},
        'rf': {'forecast': rf_pred, 'conf_int': rf_conf, 'model': rf_model},
        'linear': {'forecast': linear_pred, 'conf_int': linear_conf, 'model': linear_model}
        }
  return results

In [19]:
def create_forecast_plot_1(group, data, ax):
  historical = data['historical']

  # Plot historical data
  ax.plot(historical.index, historical.values, 'o-', label='Tarixi məlumatlar',
          color='blue', linewidth=2, markersize=4)
  # Future years
  future_years = pd.date_range(start='2024', periods=3, freq='Y')

  # Define which methods to keep for each group
  methods_config = {
      '0-13 yaşlı - cəmi': ['prophet', 'arima'],
      '14-17 yaşlı - cəmi': ['prophet', 'linear'],
      '18-29 yaşlı - cəmi': ['prophet', 'rf'], # Corrected from 'rfr' to 'rf'
      '30-44 yaşlı - cəmi': ['prophet', 'arima'],
      '45-64 yaşlı - cəmi': ['arima', 'prophet'],
      '65 və yuxarı yaşda - cəmi': ['prophet', 'linear'],
      'kişilər': ['prophet', 'arima'],
      'qadınlar': ['prophet', 'linear'],
      'İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər': ['prophet', 'rf'],
      'Əhalinin hər 100 000 nəfərinə -cəmi (müvafiq cins və yaş qruplarına görə)': ['prophet', 'linear']
      }
  # Method display names and colors
  method_info = {
      'arima': {'name': 'ARIMA', 'color': 'red'},
      'prophet': {'name': 'Prophet', 'color': 'green'},
      'rf': {'name': 'Random Forest', 'color': 'orange'},
      'linear': {'name': 'Xətti Trend', 'color': 'purple'}
      }

  # Get methods to plot for this specialty
  methods_to_plot = methods_config.get(group, ['arima', 'prophet'])

  # Plot forecasts for selected methods
  for method in methods_to_plot:
    info = method_info[method]
    forecast = data[method]['forecast']

    ax.plot(future_years, forecast, 'o--', label=f'{info["name"]} proqnozu',
            color=info['color'], linewidth=2, markersize=6)

    # Add confidence intervals if available
    conf_int = data[method]['conf_int']
    if conf_int is not None:
      try:
        if hasattr(conf_int, 'values'):
          conf_int = conf_int.values
        if conf_int.ndim == 2 and conf_int.shape[1] >= 2:
          ax.fill_between(future_years, conf_int[:, 0], conf_int[:, 1],
                          alpha=0.2, color=info['color'])
        elif conf_int.ndim == 1 and len(conf_int) >= 2:
          margin = np.std(forecast) * 0.5
          ax.fill_between(future_years, forecast - margin, forecast + margin,
                          alpha=0.2, color=info['color'])
      except:
        pass  # Skip if error

  ax.set_title(f'{group} - Zaman Seriyası Analizi və Proqnoz', fontsize=14, fontweight='bold')
  ax.set_xlabel('İl', fontsize=12)

  # Custom y-axis labels based on group
  if '100 000' in group:
    ylabel = '100 000 nəfərə düşən xəstə sayı'
  elif 'nəfər' in group:
    ylabel = 'Xəstə sayı (nəfər)'
  else:
    ylabel = 'Xəstəlik nisbəti'

  ax.set_ylabel(ylabel, fontsize=12)
  ax.legend(fontsize=10)
  ax.grid(True, alpha=0.3)

  # Add trend line
  from scipy import stats
  years_numeric = np.arange(len(historical))
  slope, intercept, r_value, p_value, std_err = stats.linregress(years_numeric, historical.values)

  # Extend trend line to future
  all_years_numeric = np.arange(len(historical) + 3)
  trend_line = slope * all_years_numeric + intercept
  all_years = list(historical.index) + list(future_years)

  ax.plot(all_years, trend_line, '--', color='gray', alpha=0.7,
            label=f'Trend (R²={r_value**2:.3f})')
  ax.legend(fontsize=9)

  # Add explanatory text
  explanations = {
      '0-13 yaşlı - cəmi': (
          "Uşaq yaş qrupunda vərəm xəstəliyi hallarının sayı 2007-2014-cü illərdə artım tendensiyası göstərmiş, "
          "2015-ci ildən etibarən isə azalmağa başlamışdır. Bu azalma uşaqlar üçün peyvənd proqramlarının genişləndirilməsi "
          "və profilaktik tədbirlərin gücləndirilməsi ilə əlaqədardır. Proqnozlar göstərir ki, gələcək illərdə bu yaş qrupunda "
          "vərəm xəstəliyi hallarının sayı sabit qalacaq və ya cüzi azalma tendensiyası davam edəcək."),

      '18-29 yaşlı - cəmi': (
          "Gənclər arasında vərəm xəstəliyi halları 2007-2010-cu illərdə pik həddə çatmış, sonradən tədricən azalmağa başlamışdır. "
          "Bu yaş qrupunda xəstəliyin yayılmasının əsas səbəbləri sıx sosial əlaqələr, immun sisteminin zəifliyi və "
          "sağlamlıq mədəniyyətinin aşağı olmasıdır. Proqnoz modelləri göstərir ki, gələcək illərdə bu qrupda xəstəlik hallarının "
          "sayında yumşaq azalma davam edəcək."),

      'İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər': (
          "Ümumi vərəm xəstəliyi hallarının sayı 2007-2011-ci illərdə artmış, 2012-2020-ci illərdə isə davamlı azalma "
          "tendensiyası müşahidə edilmişdir. COVID-19 pandemiyası dövründə (2020-2021) xəstəlik hallarında kəskin azalma "
          "qeydə alınmışdır. Bu, qarantin tədbirləri və sosial məsafənin qorunması ilə əlaqədardır. Proqnozlar göstərir ki, "
          "2024-2026-cı illərdə xəstəlik hallarının sayı təxminən sabit qalacaq.")
    }

  # Add values for each corresponding years
  for year, value in zip(historical.index, historical.values):
    ax.annotate(f'{(value/10)}K', xy=(year, value), xytext=(0, 3),
                fontsize=2, ha='right', textcoords='offset points',
                color='black', weight='bold')

  if group in explanations:
    wrapped_text = textwrap.fill(explanations[group], width=100)
    ax.text(0.5, -0.25, wrapped_text, ha='center', va='top', transform=ax.transAxes,
            fontsize=10, style='italic', color='#555555')

In [20]:
def create_tuberculosis_summary_statistics(results):
  """Create summary statistics table"""
  summary_data = []

  for group, data in results.items():
    historical = data['historical']

    # Calculate statistics
    mean_val = historical.mean()
    std_val = historical.std()
    trend = (historical.iloc[-1] - historical.iloc[0]) / len(historical) if len(historical) > 1 else 0

    # Average forecast across methods
    forecasts = []
    for method in ['arima', 'prophet', 'rf', 'linear']:
      try:
        method_forecast = data[method]['forecast']
        if hasattr(method_forecast, '__len__') and len(method_forecast) >= 3:
          forecasts.append(method_forecast)
      except:
        continue

    # Ensure there are always 3 forecast values for averaging, use last historical value if not available
    padded_forecasts = [] # Initialize padded_forecasts
    for f in forecasts:
        padded_f = list(f) + [historical.iloc[-1]] * (3 - len(f)) if len(f) < 3 else list(f)[:3]
        padded_forecasts.append(padded_f)


    if padded_forecasts:
        avg_forecast_2024 = np.mean([f[0] for f in padded_forecasts])
        avg_forecast_2025 = np.mean([f[1] for f in padded_forecasts])
        avg_forecast_2026 = np.mean([f[2] for f in padded_forecasts])
    else:
      # Fallback values if no valid forecasts available
      avg_forecast_2024 = historical.iloc[-1] if not historical.empty else 0
      avg_forecast_2025 = historical.iloc[-1] if not historical.empty else 0
      avg_forecast_2026 = historical.iloc[-1] if not historical.empty else 0

    summary_data.append({
        'Göstərici': group,
        'Orta (2007-2023)': f"{mean_val:.0f}",
        'Standart sapma': f"{std_val:.0f}",
        'İllik trend': f"{trend:.1f}",
        '2024 proqnozu': f"{avg_forecast_2024:.0f}",
        '2025 proqnozu': f"{avg_forecast_2025:.0f}",
        '2026 proqnozu': f"{avg_forecast_2026:.0f}"
        })

  return pd.DataFrame(summary_data)

In [21]:
def draw_summary_figure(pdf, total_cases, age_groups, pie_labels, pie_sizes, growth_rates):
    fig = plt.figure(figsize=(8.3, 11.7))  # A4 size
    gs = plt.GridSpec(4, 1, height_ratios=[1, 1, 1, 1])
    fig.subplots_adjust(hspace=0.5)

    # Chart 1 - Total cases over time
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(total_cases.index.year, total_cases.values,
             color='#D32F2F', linewidth=2, marker='o', markersize=4)
    ax1.set_title('İlk Dəfə Diaqnoz Qoyulmuş Vərəm Xəstələrinin Sayı (2007-2023)',
                 fontsize=12, fontweight='bold', pad=8)
    ax1.set_xlabel('İl', fontsize=9)
    ax1.set_ylabel('Xəstə sayı (nəfər)', fontsize=9)
    ax1.grid(True, alpha=0.3)
    ax1.annotate(f"{int(total_cases.values[-1])}",
                 xy=(total_cases.index.year[-1], total_cases.values[-1]),
                 xytext=(5, 0), textcoords='offset points',
                 fontsize=8, fontweight='bold', color='#D32F2F')
    ax1.tick_params(axis='both', labelsize=8)

    # Chart 2 - Age group distribution (bar)
    ax2 = fig.add_subplot(gs[1])
    bars = ax2.barh(list(age_groups.keys()), list(age_groups.values()),
                    color='#1976D2', alpha=0.85)
    ax2.set_title('2023-cü İldə Yaş Qruplarına görə Vərəm Halları',
                 fontsize=11, fontweight='bold', pad=6)
    ax2.set_xlabel('Xəstə sayı', fontsize=9)
    ax2.tick_params(axis='both', labelsize=8)
    ax2.set_xlim(0, max(age_groups.values()) * 1.2)

    for label in ax2.get_yticklabels():
        label.set_horizontalalignment('right')

    for bar in bars:
        ax2.text(bar.get_width() + max(age_groups.values()) * 0.02,
                 bar.get_y() + bar.get_height()/2,
                 f"{int(bar.get_width())}", va='center',
                 fontsize=7.5, fontweight='bold')

    # Chart 3 - Age group distribution (pie)
    ax3 = fig.add_subplot(gs[2])
    pie_colors = plt.cm.Paired(np.linspace(0, 1, len(pie_labels)))
    ax3.pie(pie_sizes, labels=pie_labels, autopct='%1.0f%%',
            startangle=140, colors=pie_colors,
            textprops={'fontsize': 8}, pctdistance=0.75, radius=0.65)
    ax3.set_title('2023-cü İl üzrə Yaş Qruplarının Paylanması',
                 fontsize=11, fontweight='bold', pad=6)

    # Chart 4 - Growth rates
    ax4 = fig.add_subplot(gs[3])
    growth_colors = ['#388E3C' if x > 0 else '#D32F2F' for x in growth_rates.values()]
    bars2 = ax4.bar(list(growth_rates.keys()), list(growth_rates.values()),
                    color=growth_colors, alpha=0.85)
    ax4.set_title('2007-2023 Dövründə Artım Dərəcəsi (%)',
                 fontsize=12, fontweight='bold', pad=6)
    ax4.set_ylabel('Artım faizi', fontsize=9)
    ax4.tick_params(axis='x', labelsize=8, rotation=30)
    ax4.tick_params(axis='y', labelsize=8)

    for bar in bars2:
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                 f"{bar.get_height():.1f}%", ha='center', va='bottom',
                 fontsize=7.5, fontweight='bold')
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)

    pdf.savefig(fig, bbox_inches='tight')
    plt.close()

In [22]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.6 MB/s eta 0:00:00


In [23]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import numpy as np
from PyPDF2 import PdfMerger
import os
import textwrap # Import textwrap

df = read_tuberculosis_data()

def generate_pdf_report(results):
  A4_SIZE = (8.3, 11.7)
  generated_pdf_path = f"{sector}_Hesabat_{end_year}.pdf"
  merged_pdf_path = "Birləşdirilmiş_Hesabat.pdf"
  title_pdf_path = f"Title_{sector}_{end_year}.pdf"
  content_pdf_path = f"Content_{sector}_{end_year}.pdf"

  # Create title page
  with PdfPages(title_pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=A4_SIZE)
    ax.axis('off')
    ax.text(0.5, 0.90, f'{sector} üzrə Statistik Hesabat',
            ha='center', fontsize=24, fontweight='bold')
    ax.text(0.5, 0.85, f'Zaman Seriyası Analizi və Proqnozlar ({start_year}-{end_year})',
            ha='center', fontsize=16)
    ax.text(0.5, 0.80, f'Hazırlayan: {name} {surname}',
            ha='center', fontsize=14)
    ax.text(0.5, 0.75, f'Tarix: {datetime.now().strftime("%d.%m.%Y")}',
            ha='center', fontsize=12)

    # Add a decorative rectangle
    rect = patches.Rectangle((0.10, 0.65), 0.80, 0.05,
                              linewidth=2, edgecolor='blue',
                              facecolor='lightblue', alpha=0.3)
    ax.add_patch(rect)

    # Add some summary information
    total_cases = results['İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər']['historical']
    last_year = total_cases.index[-1].year
    last_value = total_cases.values[-1]
    change = ((last_value - total_cases.values[0]) / total_cases.values[0]) * 100

    stats_text = f"""
    2007-{last_year} dövründə vərəm xəstəliyi hallarının sayı:
    • 2007-ci ildə: {total_cases.values[0]:.0f} nəfər
    • {last_year}-cı ildə: {last_value:.0f} nəfər
    • Dəyişiklik: {change:.1f}%
    """

    ax.text(0.5, 0.60, stats_text, ha='center', fontsize=11)

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()

  # Create content pages
  with PdfPages(content_pdf_path) as pdf:
    # Prepare data for summary figure
      total_cases = results['İlk dəfə qoyulmuş diaqnozla qeydə alınmış xəstələrin sayı- cəmi, nəfər']['historical']

      age_groups = {
          '0-13 yaş': results['0-13 yaşlı - cəmi']['historical'].iloc[-1],
          '14-17 yaş': results['14-17 yaşlı - cəmi']['historical'].iloc[-1],
          '18-29 yaş': results['18-29 yaşlı - cəmi']['historical'].iloc[-1],
          '30-44 yaş': results['30-44 yaşlı - cəmi']['historical'].iloc[-1],
          '45-64 yaş': results['45-64 yaşlı - cəmi']['historical'].iloc[-1],
          '65+ yaş': results['65 və yuxarı yaşda - cəmi']['historical'].iloc[-1]
          }

      pie_labels = list(age_groups.keys())
      pie_sizes = list(age_groups.values())

      growth_rates = {}
      for group in age_groups.keys():
        key = {
            '0-13 yaş': '0-13 yaşlı - cəmi',
            '14-17 yaş': '14-17 yaşlı - cəmi',
            '18-29 yaş': '18-29 yaşlı - cəmi',
            '30-44 yaş': '30-44 yaşlı - cəmi',
            '45-64 yaş': '45-64 yaşlı - cəmi',
            '65+ yaş': '65 və yuxarı yaşda - cəmi'}[group]

        hist = results[key]['historical']
        growth_rate = ((hist.iloc[-1] - hist.iloc[0]) / hist.iloc[0]) * 100
        growth_rates[group] = growth_rate

      # Draw summary figure
      draw_summary_figure(pdf, total_cases, age_groups, pie_labels, pie_sizes, growth_rates)

      # Add summary statistics table
      fig, ax = plt.subplots(figsize=(8.3, 11.7))
      ax.axis('off')

      summary_df = create_tuberculosis_summary_statistics(results) # Corrected function name
      table = ax.table(cellText=summary_df.values,
                      colLabels=summary_df.columns,
                      cellLoc='center',
                      loc='center',
                      colColours=['#f3f3f3']*len(summary_df.columns))

      table.auto_set_font_size(False)
      table.set_fontsize(9)
      table.scale(1, 1.8)

      ax.set_title('Statistik Göstəricilər və Proqnozlar', fontsize=14, fontweight='bold', pad=20)
      pdf.savefig(fig, bbox_inches='tight')
      plt.close()

      # Add individual forecast plots for each group
      for group, data in results.items():
        fig = plt.figure(figsize=A4_SIZE)
        ax = fig.add_axes([0.1, 0.5, 0.87, 0.45])
        create_forecast_plot_1(group, data, ax) # Corrected function name
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()

  # Merge PDFs
  from PyPDF2 import PdfMerger
  merger = PdfMerger()
  merger.append(title_pdf_path)
  merger.append(content_pdf_path)
  merger.write(merged_pdf_path)
  merger.close()

  # Clean up temporary files
  import os
  try:
    os.remove(title_pdf_path)
    os.remove(content_pdf_path)
  except:
    pass

  print(f"✅ Hesabat tamamlandı: '{merged_pdf_path}'")

# Main execution
print("PDF hesabatı yaradılır...")
results = perform_tuberculosis_analysis()
generate_pdf_report(results)

PDF hesabatı yaradılır...


INFO:prophet:n_changepoints greater than number of observations. Using 12.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwyjehrgv/up7tgzw0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwyjehrgv/3e9ebf6a.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24976', 'data', 'file=/tmp/tmpwyjehrgv/up7tgzw0.json', 'init=/tmp/tmpwyjehrgv/3e9ebf6a.json', 'output', 'file=/tmp/tmpwyjehrgv/prophet_model8xov1cr4/prophet_model-20250627165655.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:56:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:56:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 12.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwyjehrgv/1_5nsj2c.json
DEBUG:cmdstanpy:input tempfi

✅ Hesabat tamamlandı: 'Birləşdirilmiş_Hesabat.pdf'
